In [ ]:
!pip install bertopic pymorphy3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

# Скачиваем датасет из 900 новостей из постов ВКонтакте региональных сообществ Нижнего Новгорода, Саратова и Ростова

In [ ]:
!gdown 1a-eCviQVqkNPe5-oiOuAVuf04hbBOTJ4eA52dpKtsb8

Downloading...
From (original): https://drive.google.com/uc?id=1a-eCviQVqkNPe5-oiOuAVuf04hbBOTJ4eA52dpKtsb8
From (redirected): https://docs.google.com/spreadsheets/d/1a-eCviQVqkNPe5-oiOuAVuf04hbBOTJ4eA52dpKtsb8/export?format=xlsx
To: /content/vk_wall_posts.xlsx
286kB [00:00, 3.17MB/s]


# Загружаем библиотеки

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import ngrams
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('russian'))
stop_words.update(['нижний', 'новгород', 'саратов', 'ростов', 'область', 'нижегородский', 'ростовский', 'саратовский', 'апрель', 'май', 'это'])

from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

import pandas as pd
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
file_path = "/content/vk_wall_posts.xlsx"
df = pd.read_excel(file_path)
df.dropna(inplace=True)
df.head()

,post_id,post_text,post_date,post_likes,post_reposts,post_comments,owner_id
0,668205,Звание Почётного гражданина Борского округа пр...,1745919847,124,8,7,-8470759
1,668381,Подписчица заметила в трейлере нового «Просток...,1745949314,13,3,1,-8470759
2,668375,"Бор тут залетел в топ\n\nТеперь имейте в виду,...",1745947860,23,4,1,-8470759
3,668367,Канатка открывается уже послезавтра!\n\nОднако...,1745945520,30,15,5,-8470759
4,668346,Скорбная новость\n\nУшел из жизни Евгений Миха...,1745942475,62,11,2,-8470759


# Определим города по айди сообществ для удобства

In [ ]:
df['owner_id'] = df['owner_id'].astype(str)
def city_name(id):
    if id in ['-78488809', '-27566010', '-87292497']:
        return "Саратов"
    elif id in ['-104083518', '-36039', '-36578247']:
        return "Ростов"
    else:
        return "Нижний Новгород"
df['city'] = df['owner_id'].apply(city_name)

In [ ]:
df.head()

,post_id,post_text,post_date,post_likes,post_reposts,post_comments,owner_id,city
0,668205,Звание Почётного гражданина Борского округа пр...,1745919847,124,8,7,-8470759,Нижний Новгород
1,668381,Подписчица заметила в трейлере нового «Просток...,1745949314,13,3,1,-8470759,Нижний Новгород
2,668375,"Бор тут залетел в топ\n\nТеперь имейте в виду,...",1745947860,23,4,1,-8470759,Нижний Новгород
3,668367,Канатка открывается уже послезавтра!\n\nОднако...,1745945520,30,15,5,-8470759,Нижний Новгород
4,668346,Скорбная новость\n\nУшел из жизни Евгений Миха...,1745942475,62,11,2,-8470759,Нижний Новгород


# Предобработка текста: удаление всего, кроме кириллицы, токенизация и лемматизация, фильтрация стоп-слов

In [ ]:
def clean_texts(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text)  # оставляем только русские буквы
    tokens = word_tokenize(text.lower(), language='russian')
    lemmas = []
    for token in tokens:
      if morph.parse(token)[0].normal_form not in stop_words:
        lemmas.append(token)
    return ' '.join(lemmas)

In [ ]:
df['post_text'] = df['post_text'].apply(clean_texts)

In [ ]:
df.head()

,post_id,post_text,post_date,post_likes,post_reposts,post_comments,owner_id,city
0,668205,звание почётного гражданина борского округа пр...,1745919847,124,8,7,-8470759,Нижний Новгород
1,668381,подписчица заметила трейлере нового простокваш...,1745949314,13,3,1,-8470759,Нижний Новгород
2,668375,бор залетел топ имейте виду прославиться всю р...,1745947860,23,4,1,-8470759,Нижний Новгород
3,668367,канатка открывается послезавтра однако начальн...,1745945520,30,15,5,-8470759,Нижний Новгород
4,668346,скорбная новость ушел жизни евгений михайлович...,1745942475,62,11,2,-8470759,Нижний Новгород


# Делим тексты по городам

In [ ]:
def separate_into_cities(df):
    nizhny = []
    saratov = []
    rostov = []

    for index, row in df.iterrows():
      if row['city'] == 'Нижний Новгород':
        nizhny.append(row['post_text'])
      elif row['city'] == 'Саратов':
        saratov.append(row['post_text'])
      elif row['city'] == 'Ростов':
        rostov.append(row['post_text'])

    return nizhny, saratov, rostov

In [ ]:
nizhny, saratov, rostov = separate_into_cities(df)
print(nizhny[:10], saratov[:3], rostov[:3], sep='\n')

['звание почётного гражданина борского округа присвоено семёнычевой галине ивановне ветерану великой отечественной войны семенычева девичестве волкова галина ивановна родилась марта года д чистяки варначёвского сельсовета году исполнилось лет году пошла первый класс начальной школы д трубниково начальную школу класса закончила году позднее обучалась г бор окончила классов году окончания школы работала колхозе красный пахарь д чистяки полеводческой бригаде начала великой отечественной войны работала торфяных разработках борском районе года призвана военную службу служба проходила м дивизионе й отдельной зенитной артиллерийской бригады должности связиста дивизион охранял промышленные объекты г горький г бор г балахна вражеских бомбардировок года составе зенитноартиллерийского дивизиона направлена дальний восток октября года проходила службу демобилизовалась октября года демобилизации вновь работала колхозе красный пахарь бригадиром полеводческой бригады укрупнения колхоз красный пахарь п

# Готовим модель вместе с векторизатором и снижением размерности

In [ ]:
umap_model = UMAP(
    n_neighbors=5,
    n_components=3,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

In [ ]:
hdbscan_model = HDBSCAN(
    min_cluster_size=20,
    min_samples=10,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)

In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1,2))

In [ ]:
model = BERTopic(language = 'multilingual',
                 embedding_model='sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
                 vectorizer_model=vectorizer_model,
                 umap_model=umap_model,
                 hdbscan_model=hdbscan_model,
                 calculate_probabilities=True,
                 verbose=True)

# Обучение модели Бертопик

## Саратов

In [ ]:
topics, probs = model.fit_transform(saratov)

2025-05-15 16:12:21,776 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-05-15 16:12:27,669 - BERTopic - Embedding - Completed ✓
2025-05-15 16:12:27,671 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-15 16:12:28,056 - BERTopic - Dimensionality - Completed ✓
2025-05-15 16:12:28,057 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-15 16:12:28,076 - BERTopic - Cluster - Completed ✓
2025-05-15 16:12:28,079 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-15 16:12:28,188 - BERTopic - Representation - Completed ✓


In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,69,-1_рублей_кладбище_поделись подпишись_подпишись,"[рублей, кладбище, поделись подпишись, подпиши...",[сутро сегодня четверг днем местами кратковрем...
1,0,70,0_году_ул_работы_скверов,"[году, ул, работы, скверов, проектирование, со...",[саратовцы оценили проекты володина ремонту дв...
2,1,48,1_победы_года_района_место,"[победы, года, района, место, псковской, году,...",[привезут частичку вечного огня мамаева курган...
3,2,33,2_сообщили_мужчина_дтп_летний,"[сообщили, мужчина, дтп, летний, улице, рф, ук...",[энгельсе полицейский автомобиле сбил ребенка ...
4,3,30,3_фото_стариков_фото антон_антон стариков,"[фото, стариков, фото антон, антон стариков, а...","[фото юрий шатохин, фото юрий шатохин, фото ан..."
5,4,26,4_майские_праздники_среди_майские праздники,"[майские, праздники, среди, майские праздники,...",[жителей ждёт двухдневная рабочая неделя длинн...
6,5,21,5_ночью_градусов_днём_температура,"[ночью, градусов, днём, температура, местами, ...",[мокрый снег заморозки погодой встретит жителе...


In [ ]:
# Здесь можно детально посмотреть на разные топики по индексу
model.get_topic(1)

[('победы', np.float64(0.01222158806726219)),
 ('года', np.float64(0.012023023041310067)),
 ('района', np.float64(0.011497240408700597)),
 ('место', np.float64(0.010861034202995506)),
 ('псковской', np.float64(0.00946946006203122)),
 ('году', np.float64(0.009368016397195035)),
 ('участие', np.float64(0.008929121839508346)),
 ('огня', np.float64(0.008571204841284881)),
 ('инструкторкинолог', np.float64(0.008571204841284881)),
 ('ик', np.float64(0.008571204841284881))]

In [ ]:
# Визуализация топиков с индексом значимости слов
model.visualize_barchart(top_n_topics=16, n_words=8)

In [ ]:
# Матрица схожести. Показывает косинусное расстояние между топиками
model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

In [ ]:
# Интерактивная карта расстояний между топиками
model.visualize_topics()

In [ ]:
# Метод visualize_hierarchy() используется для визуализации иерархической структуры тематического моделирования.
# В частности, он отображает иерархическое дерево, в котором каждый узел представляет собой тему, а ребра между узлами обозначают иерархические отношения между темами.
model.visualize_hierarchy(top_n_topics=35)

## Нижний Новгород

In [ ]:
topics, probs = model.fit_transform(nizhny)

2025-05-15 16:13:11,412 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-05-15 16:13:12,927 - BERTopic - Embedding - Completed ✓
2025-05-15 16:13:12,929 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-15 16:13:13,320 - BERTopic - Dimensionality - Completed ✓
2025-05-15 16:13:13,321 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-15 16:13:13,342 - BERTopic - Cluster - Completed ✓
2025-05-15 16:13:13,345 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-15 16:13:13,416 - BERTopic - Representation - Completed ✓


In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,54,-1_школы_изза_около_которые,"[школы, изза, около, которые, пыльцы, остановк...",[бору перекроют движение связи проведением лег...
1,0,124,0_лет_подробнее_культурное_фестиваль,"[лет, подробнее, культурное, фестиваль, афиша,...",[планируем неделю традиционной афише собираем ...
2,1,46,1_водитель_произошло_двое_домов,"[водитель, произошло, двое, домов, словам, под...",[летнего нижегородца который насмерть сбил жен...
3,2,32,2_видео_мог_нашего_массаж,"[видео, мог, нашего, массаж, собак, яд, марта,...",[вовемя вернуля дмй выключй электпибы одя дом ...
4,3,26,3_рублей_года_млн рублей_млн,"[рублей, года, млн рублей, млн, отдела, мвд, р...",[несколько нижегородок среди которых бывшая за...
5,4,20,4_смену_профессиональной_компании_работы,"[смену, профессиональной, компании, работы, им...",[самый большой автодром фаворит города требует...


In [ ]:
model.get_topic(2)

[('видео', np.float64(0.01964799779697008)),
 ('мог', np.float64(0.017641443895446383)),
 ('нашего', np.float64(0.01518007555432635)),
 ('массаж', np.float64(0.014985506774981271)),
 ('собак', np.float64(0.014985506774981271)),
 ('яд', np.float64(0.01450536547672809)),
 ('марта', np.float64(0.014113155116357107)),
 ('люди', np.float64(0.013241315037704473)),
 ('очень', np.float64(0.012292160567880683)),
 ('котят', np.float64(0.011703498517772654))]

In [ ]:
model.visualize_barchart(top_n_topics=16, n_words=8)

In [ ]:
model.visualize_heatmap(n_clusters=4, width=1000, height=1000)

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_hierarchy(top_n_topics=35)

## Ростов

In [ ]:
topics, probs = model.fit_transform(rostov)

2025-05-15 16:16:32,011 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-05-15 16:16:33,385 - BERTopic - Embedding - Completed ✓
2025-05-15 16:16:33,386 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-15 16:16:33,950 - BERTopic - Dimensionality - Completed ✓
2025-05-15 16:16:33,953 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-15 16:16:33,982 - BERTopic - Cluster - Completed ✓
2025-05-15 16:16:33,987 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-15 16:16:34,069 - BERTopic - Representation - Completed ✓


In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,78,-1_рублей_интернет_могут_внимание,"[рублей, интернет, могут, внимание, интернета,...",[кладбище батайске нормально вообще ещё шашлык...
1,0,73,0_паблика_внимание_подпишись_подписчики,"[паблика, внимание, подпишись, подписчики, смо...",[технику ростсельмаша подготовили внимание вид...
2,1,53,1_волосы_чт_гд_пи,"[волосы, чт, гд, пи, лт, процедура, зоопарк, в...",[девушки устроили охоту сирень подписчика крас...
3,2,51,2_россии_работу_центр_лет,"[россии, работу, центр, лет, фото, год, мвд, м...",[ростовенадону открылся центр тестирования миг...
4,3,22,3_победы_фонтан_великой_года,"[победы, фонтан, великой, года, перемирие, пам...",[запустили обновлённый фонтан лиры пушкинской ...
5,4,21,4_которые_левом_видео_проблемы,"[которые, левом, видео, проблемы, выжигания, л...",[хотят массово эвуивть мшины вдитлйнарушителей...


In [ ]:
model.get_topic(4)

[('которые', np.float64(0.02085246485843798)),
 ('левом', np.float64(0.019701231523259415)),
 ('видео', np.float64(0.015627606084381088)),
 ('проблемы', np.float64(0.015420658485050481)),
 ('выжигания', np.float64(0.015420658485050481)),
 ('левом берегу', np.float64(0.015420658485050481)),
 ('пакистана', np.float64(0.015420658485050481)),
 ('штрафа', np.float64(0.015420658485050481)),
 ('дорога', np.float64(0.015420658485050481)),
 ('прямо', np.float64(0.014775923642444561))]

In [ ]:
model.visualize_barchart(n_words=8, title='Топ-слова по темам в Ростове')

In [ ]:
model.visualize_heatmap(n_clusters=4, width=1000, height=1000)

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_hierarchy(use_ctfidf=True)